In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
import statsmodels.api as sm
from   statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from sklearn.linear_model import Lasso, Ridge

import warnings 
warnings.filterwarnings('ignore')



In [ ]:
df=pd.read_csv('../input/auto-insurance-customerlifetimevalue/data.csv')
df.head()

### Expolary Data Analysis

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
df.rename(columns={'Customer Lifetime Value':'CLV'},inplace=True)


In [ ]:
df['CLV']

In [ ]:
df.info()

In [ ]:
df['Customer'].count

In [ ]:
df['Number of Open Complaints'].value_counts()

In [ ]:
df['Number of Policies'].value_counts()

- As we see there is only 5 & 9 category in columns, Number of open compliant & Number of policies. so we convert that numerical datatype into Object


In [ ]:

df['Number of Open Complaints'] = df['Number of Open Complaints'].astype('object',copy=False)

In [ ]:
df.info()

In [ ]:
list(df.select_dtypes(exclude=['object']).columns)


In [ ]:
sns.boxplot(df['Income'])

In [ ]:
sns.boxplot(df['Monthly Premium Auto'])

In [ ]:
sns.boxplot(df['Months Since Last Claim'])

In [ ]:
sns.boxplot(df['Months Since Policy Inception'])

In [ ]:
sns.boxplot(df['Total Claim Amount'])

- From above boxplot we can observed that there are outlier in column Total claim amount & monthly premium auto

### Outlier treatment

## Total cliam Amount

In [ ]:
df['Total Claim Amount'].describe()

In [ ]:
fig,ax=plt.subplots(1,3,figsize=(15,5))
sns.boxplot(df['Total Claim Amount'],ax=ax[1])

sns.distplot(df['Total Claim Amount'],ax=ax[0])
sns.scatterplot(df['Total Claim Amount'],df['CLV'],ax=ax[2])
plt.axvline(1800,color='r')

In [ ]:
sns.scatterplot(df['Total Claim Amount'],df['CLV'])
plt.axvline(1800,color='r')

In [ ]:
df[df['Total Claim Amount']>1800]['Total Claim Amount'].count()

In [ ]:
# df=df[df['Total Claim Amount']<1800]


In [ ]:
print('Data point remain',len(df))

In [ ]:
# q1=df['Total Claim Amount'].quantile(0.25)
# q2=df['Total Claim Amount'].quantile(0.50)
# q3=df['Total Claim Amount'].quantile(0.75)
# IQR=q3-q1



In [ ]:
# Upper_whiskers=q3+1.5*IQR
# Upper_whiskers

In [ ]:
# df[df['Total Claim Amount']>961]['Total Claim Amount'].count()

In [ ]:
# a=df[~((df['Total Claim Amount']<(q1-1.5*IQR))|(df['Total Claim Amount']>(q3+1.5*IQR)))]


In [ ]:
# a['Total Claim Amount'].describe()

In [ ]:
# sns.scatterplot(a['Total Claim Amount'],a['CLV'])

In [ ]:
sns.distplot(df['Total Claim Amount'])

In [ ]:
sns.boxplot(df['Total Claim Amount'])

- we do some practical to chcek whether our model work proper if we remove this outlier 

In [ ]:
sns.distplot(df['CLV'])

## Monthly Premium Auto

In [ ]:
fig,ax=plt.subplots(1,3,figsize=(15,5))
sns.boxplot(df['Monthly Premium Auto'],ax=ax[1])

sns.distplot(df['Monthly Premium Auto'],ax=ax[0])
sns.scatterplot(df['Monthly Premium Auto'],df['CLV'],ax=ax[2])
plt.axvline(204,color='r')

In [ ]:
Upper_whiskers=df['Monthly Premium Auto'].quantile(0.75)+1.5*df['Monthly Premium Auto'].quantile(0.75)-df['Monthly Premium Auto'].quantile(0.25)
Upper_whiskers

In [ ]:
df[df['Monthly Premium Auto']>204.5]['Monthly Premium Auto'].count()

In [ ]:
# q1=df['Monthly Premium Auto'].quantile(0.25)
# q2=df['Monthly Premium Auto'].quantile(0.75)
# IQR=q2-q1
# df=df[~((df['Monthly Premium Auto']<(q1-1.5*IQR))|(df['Monthly Premium Auto']>(q2+1.5*IQR)))]


In [ ]:
fig,ax=plt.subplots(1,3,figsize=(15,5))
sns.boxplot(df['Monthly Premium Auto'],ax=ax[1])

sns.distplot(df['Monthly Premium Auto'],ax=ax[0])
sns.scatterplot(df['Monthly Premium Auto'],df['CLV'],ax=ax[2])
plt.axvline(204,color='r')

### Feature Engineering

In [ ]:
df['Effective To Date']=pd.to_datetime(df['Effective To Date'],infer_datetime_format=True)
df.info()

In [ ]:
df['Months']=df['Effective To Date'].dt.month
df['Months'].value_counts()

In [ ]:
df['Months'] = df['Months'].astype('object',copy=False)

In [ ]:
df['Number of Policies'] = np.where(df['Number of Policies']>2,3,df['Number of Policies'])

In [ ]:
df['Number of Policies'].value_counts()

In [ ]:
df['Number of Policies'] = df['Number of Policies'].astype('object',copy=False)

In [ ]:
no=df[df['Response']=='No']['CLV']
yes=df[df['Response']=='Yes']['CLV']


In [ ]:
import scipy.stats as stats

In [ ]:
stats.mannwhitneyu(no,yes)

In [ ]:
import matplotlib.pyplot as plt
# plt.figure(figsize=(15,3))
sns.boxplot(df['CLV'])

In [ ]:
df['CLV'].describe()

In [ ]:
q1=df['CLV'].quantile(0.25)
q2=df['CLV'].quantile(0.50)
q3=df['CLV'].quantile(0.75)
IQR=q3-q1
print(IQR)

In [ ]:
l1=q3+1.5*IQR
l1

In [ ]:
l2=q1-1.5*IQR
l2

In [ ]:
df[df['CLV']>l1]['CLV'].count()

In [ ]:
df[df['CLV']>l1].head()

In [ ]:
q1=df['CLV'].quantile(0.25)
q2=df['CLV'].quantile(0.75)
IQR=q2-q1
df=df[~((df['CLV']<(q1-1.5*IQR))|(df['CLV']>(q2+1.5*IQR)))]


In [ ]:
df.shape

In [ ]:
sns.boxplot(df['CLV'])

In [ ]:
sns.boxplot(x=df['Coverage'],y=df['CLV'])

In [ ]:
# sns.scatterplot(df['Customer Lifetime Value'])
sns.scatterplot(y='CLV',x='Income',data=df)


In [ ]:
sns.scatterplot(y='CLV',x='Monthly Premium Auto',data=df)

In [ ]:
sns.scatterplot(y='CLV',x='Total Claim Amount',data=df)

In [ ]:
df.corr()

In [ ]:
# df=df[df['Customer Lifetime Value']<df['Customer Lifetime Value'].max()]

In [ ]:
df.head()

In [ ]:
df['CLV'].max()

- we can remove 1 data point from our depended veriable just because of outlier treatment, that maximum point meight affect our model

### statics approch to chcek which veriable is important

In [ ]:
sns.distplot(df['CLV'])

- As we see customer life time value is not normally distributed so we go with manwitney test

In [ ]:
df['State'].value_counts()

In [ ]:
cl=df[df['State']=='California']['CLV']
org=df[df['State']=='Oregon']['CLV']
ar=df[df['State']=='Arizona']['CLV']
Nv=df[df['State']=='Nevada']['CLV']
Wa=df[df['State']=='Washington']['CLV']

In [ ]:
a=stats.kruskal(cl,org,ar,Nv,Wa)


- As we pvalue is greater than aplha therfore accept H0 &
- the state will not impact to predict customer lifetime values

In [ ]:
catg=df.select_dtypes(include=['object'])
catg.drop(['Customer'],1,inplace=True)

In [ ]:
catg.head()

In [ ]:
col=list(catg.columns)
print(len(col))
col

In [ ]:
for i in col:
    print(i,'\n',df[i].value_counts(),'\n')

In [ ]:
ttest=[]
anova=[]
for i in col:
    if len(df[i].unique())<3:
        ttest.append(i)
    else:
        anova.append(i)

print(ttest)
print(anova)

- as our dependend veribale is continuous & which is not fallow normally distribution as we seen visually but we do one more test to verify it again

In [ ]:
df1=df.iloc[1:1670,:]
df1.shape
print(stats.shapiro(df1['CLV']))
# print(stats.jarque_bera(df['Months Since Last Claim']))

- As pvalue is less tha alpha, which we can consider 5% so therefore we reject H0 and we conculed that our traget verible is     not normally distributed

- so therefore we go with non-parametric test

In [ ]:
num=df.select_dtypes(exclude=['object'])
num.columns

In [ ]:
yes=df[df['Response']=='No']['CLV']
no=df[df['Response']=='Yes']['CLV']



In [ ]:
b=stats.mannwhitneyu(yes,no)
b[1]

In [ ]:
female=df[df['Gender']=='F']['CLV']
male=df[df['Gender']=='M']['CLV']

In [ ]:
f=stats.mannwhitneyu(female,male)

- we test above two features with non-Paramteric unpaired ttest and  we concluded that those 2 faetures is not significant 
  to predict traget 

### 1 way anova

In [ ]:
anova

In [ ]:
df['State'].value_counts()

In [ ]:
ca=df[df['State']=='California']['CLV']
Or=df[df['State']=='Oregon']['CLV']
Ar=df[df['State']=='Arizona']['CLV']
Ne=df[df['State']=='Nevada']['CLV']
Wa=df[df['State']=='Washington']['CLV']

In [ ]:
stats.kruskal(ca,Or,Ar,Ne,Wa)


In [ ]:
df['Coverage'].value_counts()

In [ ]:
Ba=df[df['Coverage']=='Basic']['CLV']
Ex=df[df['Coverage']=='Extended']['CLV']
Pr=df[df['Coverage']=='Premium']['CLV']

In [ ]:
c=stats.kruskal(Ba,Ex,Pr)

In [ ]:
df['Education'].value_counts()

In [ ]:
Ba=df[df['Education']=='Bachelor']['CLV']
Co=df[df['Education']=='College']['CLV']
Hi=df[df['Education']=='High School or Below']['CLV']
Ma=df[df['Education']=='Master']['CLV']
Da=df[df['Education']=='Doctor']['CLV']

In [ ]:
d=stats.kruskal(Ba,Co,Hi,Ma,Da)

In [ ]:
sns.boxplot(y='CLV',x='Education',data=df)

In [ ]:

df['EmploymentStatus'].value_counts()

In [ ]:
Ba=df[df['EmploymentStatus']=='Employed']['CLV']
Co=df[df['EmploymentStatus']=='Unemployed']['CLV']
Hi=df[df['EmploymentStatus']=='Medical Leave']['CLV']
Ma=df[df['EmploymentStatus']=='Disabled']['CLV']
Da=df[df['EmploymentStatus']=='Retired']['CLV']

In [ ]:
e=stats.kruskal(Ba,Co,Hi,Ma,Da)

In [ ]:
df['Location Code'].value_counts()

In [ ]:
Ba=df[df['Location Code']=='Suburban']['CLV']
Co=df[df['Location Code']=='Rural']['CLV']
Hi=df[df['Location Code']=='Urban']['CLV']

In [ ]:
g=stats.kruskal(Ba,Co,Hi)

In [ ]:
df['Marital Status'].value_counts()

In [ ]:
Ba=df[df['Marital Status']=='Married']['CLV']
Co=df[df['Marital Status']=='Single']['CLV']
Hi=df[df['Marital Status']=='Divorced']['CLV']

In [ ]:
h=stats.kruskal(Ba,Co,Hi)

In [ ]:
df['Number of Open Complaints'].value_counts()

In [ ]:
Ba=df[df['Number of Open Complaints']==0]['CLV']
Co=df[df['Number of Open Complaints']==1]['CLV']
Hi=df[df['Number of Open Complaints']==2]['CLV']
Ma=df[df['Number of Open Complaints']==3]['CLV']
Da=df[df['Number of Open Complaints']==4]['CLV']
As=df[df['Number of Open Complaints']==5]['CLV']

In [ ]:
i=stats.kruskal(Ba,Co,Hi,Ma,Da,As)
e

In [ ]:
df['Number of Policies'].value_counts()

In [ ]:
Ba=df[df['Number of Policies']==1]['CLV']
Co=df[df['Number of Policies']==2]['CLV']
Hi=df[df['Number of Policies']==3]['CLV']


In [ ]:
j=stats.kruskal(Ba,Co,Hi)
j

In [ ]:
df['Policy Type'].value_counts()

In [ ]:
Ba=df[df['Policy Type']=='Personal Auto']['CLV']
Co=df[df['Policy Type']=='Corporate Auto']['CLV']
Hi=df[df['Policy Type']=='Special Auto']['CLV']

In [ ]:
k=stats.kruskal(Ba,Co,Hi)

In [ ]:
df['Policy'].value_counts()

In [ ]:
Ba=df[df['Policy']=='Personal L3']['CLV']
Co=df[df['Policy']=='Personal L2']['CLV']
Hi=df[df['Policy']=='Personal L1']['CLV']
Ma=df[df['Policy']=='Corporate L3']['CLV']
Da=df[df['Policy']=='Corporate L2']['CLV']
Ca=df[df['Policy']=='Corporate L1']['CLV']
s1=df[df['Policy']=='Special L2']['CLV']
s2=df[df['Policy']=='Special L3']['CLV']
s3=df[df['Policy']=='Special L1']['CLV']

In [ ]:
l=stats.kruskal(Ba,Co,Hi,Ma,Da,Ca,s1,s2,s3)

In [ ]:
df['Renew Offer Type'].value_counts()

In [ ]:
Ba=df[df['Renew Offer Type']=='Offer1']['CLV']
Co=df[df['Renew Offer Type']=='Offer2']['CLV']
Hi=df[df['Renew Offer Type']=='Offer3']['CLV']
Ma=df[df['Renew Offer Type']=='Offer4']['CLV']

In [ ]:
m=stats.kruskal(Ba,Co,Hi,Ma)

In [ ]:
df['Sales Channel'].value_counts()

In [ ]:
Ba=df[df['Sales Channel']=='Agent']['CLV']
Co=df[df['Sales Channel']=='Branch']['CLV']
Hi=df[df['Sales Channel']=='Call Center']['CLV']
Ma=df[df['Sales Channel']=='Web']['CLV']

In [ ]:
n=stats.kruskal(Ba,Co,Hi,Ma)

In [ ]:
anova

In [ ]:
df['Vehicle Class'].value_counts()

In [ ]:
Ba=df[df['Vehicle Class']=='Four-Door Car']['CLV']
Co=df[df['Vehicle Class']=='Two-Door Car']['CLV']
Hi=df[df['Vehicle Class']=='SUV']['CLV']
Ma=df[df['Vehicle Class']=='Sports Car']['CLV']
Da=df[df['Vehicle Class']=='Luxury SUV']['CLV']
Ca=df[df['Vehicle Class']=='Luxury Car']['CLV']

In [ ]:
o=stats.kruskal(Ba,Co,Hi,Ma,Da,Ca)
o

In [ ]:
df['Vehicle Size'].value_counts()

In [ ]:
Ba=df[df['Vehicle Size']=='Medsize']['CLV']
Co=df[df['Vehicle Size']=='Small']['CLV']
Hi=df[df['Vehicle Size']=='Large']['CLV']

In [ ]:
p=stats.kruskal(Ba,Co,Hi)
p[1]

In [ ]:
df['Months'].value_counts()

In [ ]:
Ba=df[df['Months']==1]['CLV']
Co=df[df['Months']==2]['CLV']

In [ ]:
q=stats.kruskal(Ba,Co)
q

In [ ]:
z=[a[1],b[1],c[1],d[1],e[1],f[1],g[1],h[1],i[1],j[1],k[1],l[1],m[1],n[1],o[1],p[1],q[1]]
print(len(z))
z

In [ ]:
print(len(col))
col



In [ ]:
df2=pd.DataFrame({'columns':col,'P_vlaue':z})
df2

In [ ]:
df2['status']=df2['P_vlaue'].map(lambda x:'significant' if x<0.05 else 'not significant')
df2

In [ ]:
sns.barplot(df['State'],df['CLV'])


In [ ]:
sns.barplot(df['Response'],df['CLV'])



In [ ]:
sns.barplot(df['Gender'],df['CLV'])

In [ ]:
sns.barplot(df['Location Code'],df['CLV'])

In [ ]:
sns.barplot(df['Policy Type'],df['CLV'])

In [ ]:
sns.barplot(df['Policy'],df['CLV'])

In [ ]:
sns.barplot(df['Renew Offer Type'],df['CLV'])

In [ ]:
sns.barplot(df['Months'],df['CLV'])

### Continues veriable

In [ ]:

num=df.select_dtypes(exclude=['object'])
num=num.drop('Effective To Date',1)
num.head()

In [ ]:
X=num.drop(['CLV'],axis=1)
y=num['CLV']
# xc=sm.add_constant(x)
lin_reg=sm.OLS(y,X).fit()
lin_reg.summary()                                                                     



In [ ]:
a={'Numerical_column':['Income','Monthly Premium Auto','Months Since Last Claim','Months Since Policy Inception','Total Claim Amount'],
  'P_value':[0.011,0.000,0.102,0.483,0.057],'Status':['Significant','Significant','Not_Significant','Not_Significant','Significant']}

In [ ]:
m=pd.DataFrame(a)
m

- from above observation we can say that except columns,months since last claim & months since policy inception are not signoificant in continuous nature

In [ ]:
num.head()

In [ ]:
catg=df.select_dtypes(include=['object'])
catg=catg.drop('Customer',1)
catg.head()

In [ ]:
df1=pd.concat([catg,num],axis=1)
df.head()

In [ ]:
catg=pd.get_dummies(catg,drop_first=True)
catg.head()



In [ ]:
df=pd.concat([catg,num],axis=1)
df.head()

In [ ]:
X=df.drop(['CLV'],axis=1)
y=num['CLV']
xc=sm.add_constant(X)
lin_reg=sm.OLS(y,xc).fit()
lin_reg.summary()

### ASSUMPTIONS OF LINEAR REGRESSION.
### Linearity

In [ ]:
import statsmodels.api as sm
sm.stats.diagnostic.linear_rainbow(res=lin_reg, frac=0.5)

In [ ]:
y_pred=lin_reg.predict()

fig,ax=plt.subplots(figsize=(7,5))
sns.regplot(x=y_pred,y=y,lowess=True,line_kws={'color':'red'})
ax.set_title('observed vs predicted')
ax.set(xlabel='Predicted',ylabel='Observed')
plt.show()

In [ ]:
residuals = y_pred-y
mean_of_residuals = np.mean(residuals)
print(f"The mean of the residuals is {mean_of_residuals}")

- As mean of residual is not so much large so we can consider that linearity is present

- AS we got p_value greter than aplha(0.05) & we can also see visually that our data is linear

### Homoscedasticity_test

In [ ]:
y_pred=lin_reg.predict()
resids=lin_reg.resid
fig,ax=plt.subplots()
sns.regplot(y_pred,resids,lowess=True,line_kws={'color':'red'})  ## lowess kind of linear rerrg.
ax.set_title('residual vs predicted')
ax.set(xlabel='prediacted',ylabel='residual')
plt.show()

In [ ]:
import statsmodels.stats.api as sms
name=['F-statistic','p-value']
test=sms.het_goldfeldquandt(lin_reg.resid,lin_reg.model.exog)

test


We can also use two statistical tests: Breusch-Pagan and Goldfeld-Quandt. In both of them the null hypothesis assumes homoscedasticity and a p-value below a certain level (like 0.05) indicates we should reject the null in favor of heteroscedasticity.

Here, p value is less than 0.05 so, it is homoscedasticity distribution.

𝐻0:𝜎𝑢𝑖 𝑖𝑠 𝑐𝑜𝑛𝑠𝑡𝑎𝑛𝑡 𝑎𝑐𝑟𝑜𝑠𝑠 𝑡ℎ𝑒 𝑟𝑎𝑛𝑔𝑒 𝑜𝑓 𝑑𝑎𝑡𝑎
 
𝐻𝑎:𝜎𝑢𝑖 𝑖𝑠 𝑛𝑜𝑡 𝑐𝑜𝑛𝑠𝑡𝑎𝑛𝑡 𝑎𝑐𝑟𝑜𝑠𝑠 𝑡ℎ𝑒 𝑟𝑎𝑛𝑔𝑒 𝑜𝑓 𝑑𝑎𝑡𝑎

- goldfedquandt test fro homoscadestticity.since prob is much lower than aplha we reject the null hypothesis and accept H1   that residual are hetroscadictic

### Test of normality of residuals

In [ ]:
plt.figure(figsize=(7,5))
p = sns.distplot(residuals,kde=True)
p = plt.title('Normality of error terms/residuals')

In [ ]:
from scipy import stats
print(stats.jarque_bera(lin_reg.resid))

- As we can see p_value less than aphla that means our Null hypothesis is rejected 
- Thus we reject the null hypothesis that the error terms are normally distributed.

### No Autocorrelation

1) No Auto correlation.

Test needed : Durbin- Watson Test.

It's value ranges from 0-4. If the value of Durbin- Watson is Between 0-2, it's known as Positive Autocorrelation.
If the value ranges from 2-4, it is known as Negative autocorrelation.
If the value is exactly 2, it means No Autocorrelation.
For a good linear model, it should have low or no autocorrelation.
we can see here the values of dublin watson test (test for normality): 1.240 (POSITIVE AUTO-CORRELATION)

From the graph below, we can easily see that there is somewhat Positive autocorrelation.

In [ ]:
from statsmodels.stats.stattools import durbin_watson
durbin_watson(lin_reg.resid)

In [ ]:
import statsmodels.tsa.api as smt
plt.figure(figsize=(7,5))
acf=smt.graphics.plot_acf(lin_reg.resid,lags=40)

- As we can see that Durbin_watson value is 1.955 so there is no autocorrelation found 

### NO MULTICOLLINEARITY

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
a=pd.DataFrame({'vif': vif}, index=X.columns)
a

In [ ]:
df.head()


In [ ]:
df.shape

- we can see that there is high VIF in column Policy & policy type
- Also we have seen in statistical apprche these veriable is not significant to predict target 
- hence from above 2 conclusion we can simply remove them

In [ ]:
X=df.drop(['CLV','Policy Type_Personal Auto','Policy Type_Special Auto','Policy_Personal L1','Policy_Personal L2',
'Policy_Personal L3',
'Policy_Special L1',
'Policy_Special L2',
'Policy_Special L3',],axis=1)
y=df['CLV']
# xc=sm.add_constant(x)
lin_reg=sm.OLS(y,X).fit()
lin_reg.summary()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

In [ ]:
sns.distplot(df['CLV'])

In [ ]:
a=np.log(df['CLV'])
sns.distplot(a)

In [ ]:
b = df['CLV'].transform(lambda X: 1/X)
sns.distplot(b)
b

In [ ]:
b = df['CLV'].transform(lambda X:X**0.10)
sns.distplot(b)

In [ ]:
sns.boxplot(df['Monthly Premium Auto'])

In [ ]:
sns.distplot(df['Monthly Premium Auto'])

In [ ]:
a=np.log(df['Monthly Premium Auto'])
sns.distplot(a)

In [ ]:
b = df['Monthly Premium Auto'].transform(lambda X:X**0.1)
sns.distplot(b)

In [ ]:
X=df.drop(['CLV','Policy Type_Personal Auto','Policy Type_Special Auto','Policy_Personal L1','Policy_Personal L2','Policy_Personal L3',
'Policy_Special L1','Policy_Special L2','Policy_Special L3'],1)
y=df['CLV']          
y =np.log(y)
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.30,random_state=42)

In [ ]:
lr=LinearRegression()
lr.fit(x_train,y_train)

y_pred=lr.predict(x_test)

In [ ]:

print('-----Log---- ')
print('RMES:',np.sqrt(mean_squared_error(y_test,y_pred)))
print('R-squared:',r2_score(y_test,y_pred)) 
print('After Anti Log')
print('RMSE : ',np.sqrt(mean_squared_error(np.exp(y_test),np.exp(y_pred))))
print('r2_score : ',r2_score(np.exp(y_test),np.exp(y_pred)))


In [ ]:
print('After Anti Log')
print('RMSE : ',np.sqrt(mean_squared_error(np.exp(y_test),np.exp(y_pred))))
print('r2_score : ',r2_score(np.exp(y_test),np.exp(y_pred)))

In [ ]:
y_pred_train=lr.predict(x_train)
print('RMES:',np.sqrt(mean_squared_error(y_train,y_pred_train)))
print('R-squared:',r2_score(y_train,y_pred_train)) 

In [ ]:
X.head()

## RFE

In [ ]:
from sklearn.feature_selection import RFE


In [ ]:
model=LinearRegression()
X=df.drop(['CLV','Policy Type_Personal Auto','Policy Type_Special Auto','Policy_Personal L1','Policy_Personal L2','Policy_Personal L3',
'Policy_Special L1','Policy_Special L2','Policy_Special L3'],1)
y=df['CLV']

### initilize rfe
rfe=RFE(model,10)
rfe.fit(x_train,y_train)


print(rfe.support_)
print(rfe.ranking_)

In [ ]:
#no of features
nof_list=np.arange(1,48)            
high_score=0
#Variable to store the optimum features
nof=0           
score_list =[]
for n in range(len(nof_list)):
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 0)
    model = LinearRegression()
    rfe = RFE(model,nof_list[n])
    X_train_rfe = rfe.fit_transform(X_train,y_train)
    X_test_rfe = rfe.transform(X_test)
    model.fit(X_train_rfe,y_train)
    score = model.score(X_test_rfe,y_test)
    score_list.append(score)
    if(score>high_score):
        high_score = score
        nof = nof_list[n]
print("Optimum number of features: %d" %nof)
print("Score with %d features: %f" % (nof, high_score))

In [ ]:


X=df.drop(['CLV','Policy Type_Personal Auto','Policy Type_Special Auto','Policy_Personal L1','Policy_Personal L2','Policy_Personal L3',
'Policy_Special L1','Policy_Special L2','Policy_Special L3'],1)
y=df['CLV']
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.30,random_state=42)
nof_cols=np.arange(1,49)

model=LinearRegression()
score_list=[]

for n in range(48):
    rfe=RFE(model,n+1)
    rfe.fit(x_train,y_train)
    y_pred=rfe.predict(x_test)
    score=r2_score(y_test,y_pred)
    score_list.append(score)

In [ ]:
plt.plot(nof_cols,score_list)
plt.show()

In [ ]:
# plt.plot(nof_cols,score_list)
rfe=RFE(model,43)
rfe.fit(x_train,y_train)
pd.DataFrame(list(zip(X.columns,rfe.support_,rfe.ranking_)),columns=['col','select','rank'])

In [ ]:
y_pred=rfe.predict(x_test)
print('R2:',r2_score(y_test,y_pred))
print('Rmse:',np.sqrt(mean_squared_error(y_test,y_pred)))

## Decision Tree

In [ ]:
df1.head()

In [ ]:
catg=df1.select_dtypes(include=['object'])
catg.head()

In [ ]:
num1=df1.select_dtypes(exclude=['object'])
num1.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder


In [ ]:
cat=catg.apply(LabelEncoder().fit_transform)
cat.head()

In [ ]:
df1=pd.concat([cat,num1],axis=1)
df1.head()

In [ ]:
X=df1.drop(['CLV','State','Response','Gender','Location Code','Policy Type','Months','Policy','Renew Offer Type','Months Since Policy Inception'],1)
y=df1['CLV']
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.30,random_state=42)

In [ ]:
dt = DecisionTreeRegressor()
dt.fit(x_train,y_train)


y_pred=dt.predict(x_test)

In [ ]:
print('RMES:',np.sqrt(mean_squared_error(y_test,y_pred)))
print('R-squared:',r2_score(y_test,y_pred)) 

In [ ]:
from scipy.stats import zscore

num2=num1.apply(zscore)
num2.head()

In [ ]:
sns.distplot(num2['Monthly Premium Auto'])

In [ ]:
df2=pd.concat([cat,num2],axis=1)
df2.head()

In [ ]:
X=df2.drop(['CLV','State','Response','Gender','Location Code','Policy Type','Months'],1)
y=df2['CLV']
# y=np.log(y)
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.30,random_state=42)

In [ ]:
dt = DecisionTreeRegressor()
dt.fit(x_train,y_train)

y_pred=dt.predict(x_test)

In [ ]:
print('RMES:',np.sqrt(mean_squared_error(y_test,y_pred)))
print('R-squared:',r2_score(y_test,y_pred)) 

### Random Forest

In [ ]:
rf = RandomForestRegressor()

rf.fit(x_train,y_train)

y_pred=rf.predict(x_test)
print('RMES:',np.sqrt(mean_squared_error(y_test,y_pred)))
print('R-squared:',r2_score(y_test,y_pred)) 

## Random Forest with hyperparameter

In [ ]:
from sklearn.model_selection import GridSearchCV
rf = RandomForestRegressor()
params = {
        'max_depth' : [10,20,30],
        
        'n_estimators' : [100,200,50],
        
        "bootstrap" : [True, False],
    
        'max_features': ['auto', 'sqrt', 'log2']
        
        }

grid = GridSearchCV(estimator = rf, param_grid=params, cv = 5, n_jobs = -1, return_train_score = True )
grid.fit(x_train,y_train)
grid.best_params_

In [ ]:
rf = RandomForestRegressor(bootstrap= True,max_depth= 30,max_features= 'auto',n_estimators= 300)

rf.fit(x_train,y_train)

y_pred=rf.predict(x_test)
print('RMES:',np.sqrt(mean_squared_error(y_test,y_pred)))
print('R-squared:',r2_score(y_test,y_pred)) 

In [ ]:
from sklearn.ensemble import BaggingRegressor,GradientBoostingRegressor 

In [ ]:
br = BaggingRegressor()

br.fit(x_train,y_train)

y_pred=br.predict(x_test)
print('RMES:',np.sqrt(mean_squared_error(y_test,y_pred)))
print('R-squared:',r2_score(y_test,y_pred))

In [ ]:
br = GradientBoostingRegressor(learning_rate=0.1,n_estimators=200)

br.fit(x_train,y_train)

y_pred=br.predict(x_test)
print('RMES:',np.sqrt(mean_squared_error(y_test,y_pred)))
print('R-squared:',r2_score(y_test,y_pred))

In [ ]:
X=df1.drop(['CLV','State','Response','Gender','Location Code','Policy'],axis=1)
y=df1['CLV']
# xc=sm.add_constant(x)
lin_reg=sm.OLS(y,X).fit()
lin_reg.summary()

In [ ]:
num1.head()


In [ ]:
num3=num1[['Income','Monthly Premium Auto','Months Since Last Claim','Months Since Policy Inception','Total Claim Amount','CLV']]
num3.head()

In [ ]:
cat.head()

In [ ]:
df4=pd.concat([cat,num3],axis=1)
df4.head()

In [ ]:
n_iterations = 10
n_size = int(len(df4) * 0.50)
values = df4.values

In [ ]:
from sklearn.utils import resample

# run bootstrap
stats = list()
for i in range(n_iterations):
    # prepare train and test sets
    train = resample(values, n_samples=n_size)
    test = np.array([x for x in values if x.tolist() not in train.tolist()])
    # fit model
    model =RandomForestRegressor()
    model.fit(train[:,:-1], train[:,-1])
    # evaluate model
    predictions = model.predict(test[:,:-1])
    score = r2_score(test[:,-1], predictions)
    print(score)
    stats.append(score)

In [ ]:
plt.hist(stats)
plt.show()
# confidence intervals
alpha = 0.95
p = ((1.0-alpha)/2.0) * 100
lower = max(0.0, np.percentile(stats, p))
p = (alpha+((1.0-alpha)/2.0)) * 100
upper = min(1.0, np.percentile(stats, p))
print('%.1f confidence interval %.1f%% and %.1f%%' % (alpha*100, lower*100, upper*100))